In [19]:
# Read input from the benchmark file
def read_input(file_path):
    with open(file_path, 'r') as file:
        num_objects = int(file.readline().strip())
        bin_capacity = int(file.readline().strip())
        weights = [int(line.strip()) for line in file]

    return num_objects, bin_capacity, weights

# Example usage:
#file_path = 'benchmark/BPP_100_100_0.1_0.7_0.txt'
#file_path = 'benchmark/unordered_benchmark50.txt'
file_path = 'benchmark/benchmark500.txt'
#file_path = 'benchmark/benchmark50.txt'
num_objects, bin_capacity, weights = read_input(file_path)

**First step :** Define the heuristic that generates the first solution (here we used first fir decreasing)


In [13]:
import time

def first_fit(items, capacite_bin):
    start_time = time.time()

    bins = []
    for item in items:
        assigne = False

        for bin in bins:
            if item <= bin['espace_libre']:
                bin['items'].append(item)
                bin['espace_libre'] -= item
                assigne = True
                break

        if not assigne:
            nouveau_bin = {'espace_libre': capacite_bin - item, 'items': [item]}
            bins.append(nouveau_bin)

    execution_time = time.time() - start_time

    return bins

def first_fit_decreasing(items, capacite_bin):
    items.sort(reverse=True)  # trier les éléments par taille, du plus grand au plus petit
    result=first_fit(items, capacite_bin)
    return result

In [14]:
def evaluate_solution(bins):
    # Evaluate the solution (objective function)
    # Objective: Minimize the number of bins used
    return len(bins)

In [15]:
def generate_neighbors(bins):
    # Generate neighboring solutions by swapping items between bins
    neighbors = []
    for i in range(len(bins)):
        for j in range(i + 1, len(bins)):
            for item_i in range(len(bins[i]['items'])):
                for item_j in range(len(bins[j]['items'])):
                    neighbor = [bin.copy() for bin in bins]  # Copy current solution
                    # Swap items between bins
                    neighbor[i]['items'].append(neighbor[j]['items'].pop(item_j))
                    neighbor[j]['items'].append(neighbor[i]['items'].pop(item_i))
                    neighbors.append(neighbor)
    return neighbors

**Second step:** Define the Tabou search using the Best move as neihborhood selection strategy

In [9]:
def tabu_search(items, bin_capacity, iterations, tabu_size):
    start_time = time.time()
    current_solution = first_fit_decreasing(items,bin_capacity)
    best_solution = current_solution[:]
    tabu_list = []

    for _ in range(iterations):
        neighbors = generate_neighbors(current_solution)
        best_neighbor = None
        best_neighbor_value = float('inf')

        for neighbor in neighbors:
            if neighbor not in tabu_list:
                neighbor_value = evaluate_solution(neighbor)
                if neighbor_value < best_neighbor_value:
                    best_neighbor = neighbor
                    best_neighbor_value = neighbor_value

        if best_neighbor is None:
            break

        current_solution = best_neighbor[:]
        if evaluate_solution(current_solution) < evaluate_solution(best_solution):
            best_solution = current_solution[:]

        tabu_list.append(best_neighbor)
        if len(tabu_list) > tabu_size:
            tabu_list.pop(0)  # Remove oldest entry from tabu list
        execution_time = time.time() - start_time

    return best_solution, execution_time



In [1]:

best_solution, execution_time = tabu_search(weights, bin_capacity, iterations=100, tabu_size=10)
print("Best solution found:", best_solution)
print("Number of bins used:", evaluate_solution(best_solution))
print("Execution time:", execution_time, "seconds")

NameError: name 'tabu_search' is not defined